In [1]:
import scipy
import numpy as np
from itertools import product

In [2]:
shipping_cost = np.array([[1,3],[3,2],[2,2]]).reshape(3,2)
production_cost = np.array([6,5]).reshape(1,2)
given_demand = np.array([[50,60,75],[75,90,100],[60,75,90]]).reshape(3,3)
produce_limit = np.array([250,250]).reshape(2,1)

In [3]:
num_senario = 1
num_dest_senario = 1

In [4]:
# Shortfall Matrix 
Shortfall_lft = np.eye(num_senario*num_dest_senario*3)

# Allocation Matrix
Resource_lft = 0.5*np.repeat([[1,1,0,0,0,0],[0,0,1,1,0,0],[0,0,0,0,1,1]],1,axis=0)

# Demand Matrix
demand_rgt = np.array([75,100,90]).reshape(3,1)

# Production Matrix
Production_lft = np.repeat([[0,0],[0,0],[0,0]],1,axis=0)

# Constraint Matrix
A1_ub = -1*np.concatenate((Production_lft,Resource_lft,Shortfall_lft),axis=1)
b1_ub = -1*demand_rgt

In [5]:
# Balance Matrix
Balance_lft = np.array([[-1,0,1,0,1,0,1,0],[0,-1,0,1,0,1,0,1]])
Dummy_lft = np.zeros((2,len(Shortfall_lft)))

# Concatenate
A2_ub = np.concatenate((Balance_lft,Dummy_lft),axis=1)
b2_ub = np.array([0,0]).reshape(2,1)

In [6]:
A_ub = np.concatenate((A1_ub,A2_ub),axis=0)
b_ub = np.concatenate((b1_ub,b2_ub),axis=0)

In [7]:
obj =[]
bound = []

for i in production_cost:
    obj.extend(i)

for i in produce_limit:
    bound.append([0,i[0]])

for i in shipping_cost:
    for j in i:
        obj.extend([j])
        bound.append([0,250])

for i in range(3*num_senario*num_dest_senario):
    obj.extend([40/num_dest_senario*num_senario])
    bound.append([0,float("inf")])

In [8]:
obj = scipy.optimize.linprog(c=obj, A_ub=A_ub, b_ub=b_ub, bounds=bound, method='highs')

In [9]:
obj

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 4200.0
              x: [ 2.500e+02  2.500e+02  1.500e+02  0.000e+00  0.000e+00
                   1.700e+02  1.000e+02  8.000e+01  0.000e+00  1.500e+01
                   0.000e+00]
            nit: 7
          lower:  residual: [ 2.500e+02  2.500e+02  1.500e+02  0.000e+00
                              0.000e+00  1.700e+02  1.000e+02  8.000e+01
                              0.000e+00  1.500e+01  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  2.000e+00
                              1.000e+00  0.000e+00  0.000e+00  0.000e+00
                              2.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [ 0.000e+00  0.000e+00  1.000e+02  2.500e+02
                              2.500e+02  8.000e+01  1.500e+02  1.700e+02
                                    inf        inf        inf]
                 marginals: [-1.

In [ ]:
obj.x[2:8] 

array([250., 250., 150.,   0.,   0., 170., 100.,  80.,   0.,  15.,   0.])

In [91]:
obj.x[2:8]

array([150.,   0.,   0., 170., 100.,  80.])

In [94]:
production = obj.x[:2]
shipping_cost = obj.x[2:8]

In [93]:
print("Optimal Production: ", production)
print("Optimal Shipping Cost: ", shipping_cost)

Optimal Production:  [250. 250.]
Optimal Shipping Cost:  [150.   0.   0. 170. 100.  80.]


In [10]:
250*6 + 250*5

2750